contexto

#### Pregunta: 

¿Cómo ha evolucionado la demanda de habilidades en el mercado laboral desde la
introducción generalizada de la IA, y qué nuevas competencias han surgido como las
más valoradas?

Dataset a usar, descripcion de sus columnas y de su utilidad

In [19]:
import pandas as pd
import os

#lectura del dataset "data\tech_layoffs.xlsx" (se requiere descargar openpyxl) y "data\IA_impact_on_jobs.csv"
df_ia_impact = pd.read_csv(os.path.join("data","IA_impact_on_jobs.csv"))
df_layoffs = pd.read_csv(os.path.join("data","tech_layoffs_Q2_2024.csv"))
df_layoffs


,#,Company,Location_HQ,Region,State,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in__mil,Year,latitude,longitude
0,1,Tamara Mellon,Los Angeles,NaN,California,USA,North America,20.0,2020-03-12,"40,0",50,30,Retail,Series C,90.0,2020,34.053691,-118.242766
1,2,HopSkipDrive,Los Angeles,NaN,California,USA,North America,8.0,2020-03-13,"10,0",80,72,Transportation,Unknown,45.0,2020,34.053691,-118.242766
2,3,Panda Squad,San Francisco,San Francisco Bay Area,California,USA,North America,6.0,2020-03-13,"75,0",8,2,Consumer,Seed,1.0,2020,37.779259,-122.419329
3,4,Help.com,Austin,NaN,NaN,USA,North America,16.0,2020-03-16,"100,0",16,0,Support,Seed,6.0,2020,30.271129,-97.743700
4,5,Inspirato,Denver,NaN,NaN,USA,North America,130.0,2020-03-16,"22,0",591,461,Travel,Series C,79.0,2020,39.739236,-104.984862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,1835,Moxion Power,Richmond,San Francisco Bay Area,California,USA,North America,101.0,2024-06-26,NaN,NaN,NaN,Energy,Unknown,124.0,2024,37.538509,-77.434280
1835,1836,Perion,Tel Aviv,NaN,NaN,Israel,Asia,35.0,2024-06-26,"5,0",700,665,Marketing,Post-IPO,76.0,2024,32.085300,34.781806
1836,1837,Planet,San Francisco,San Francisco Bay Area,California,USA,North America,180.0,2024-06-26,"17,0",1059,879,Aerospace,Post-IPO,574.0,2024,37.779259,-122.419329
1837,1838,RealPage,Dallas,NaN,NaN,USA,North America,20.0,2024-06-27,"4,0",500,480,Real Estate,Acquired,105.0,2024,32.776272,-96.796856


,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.971940,77.593690
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.818410,34.988500
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.548270,-121.988570
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18,10.0,1000.0,900.0,Retail,Unknown,NaN,2023,12.971940,77.593690
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.774930,-122.419420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,3533,TomTom,Lodz,Poland,Europe,45.0,2023-12-12,NaN,NaN,NaN,Transport…,Post-IPO,NaN,2024,51.768732,19.456991
1668,3534,Chipper Cash,San Francisco Bay Area,USA,North America,15.0,2023-12-11,33.0,45.0,30.0,Finance,Series C,$302,2024,37.774930,-122.419420
1669,3535,Jungle Scout,Austin,USA,North America,NaN,2023-12-09,NaN,NaN,NaN,Retail,Private …,$110,2024,30.267150,-97.743060
1670,3536,SmileDirectClub,Nashville,USA,North America,NaN,2023-12-09,100.0,NaN,0.0,Healthcare,Post-IPO,$694,2024,36.165890,-86.784440
